In [137]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.gridspec as gridspec
import contextily as ctx

%matplotlib qt

In [138]:
class Application():
    def __init__(self):
        self.Q = 0.01                              # increase in Q2 discharge from proposed development (cfs)
        self.I = 0.1                              # increase impervious area from proposed development (acres)
        self.impact_threshold = 0.1               # threshold to determine extent of downstream impact (% of instream discharge)
        
        
        self.last_pick = None
        self.df = gpd.read_file(r'/CWS_pilot_reaches_final_draft_04042023.geojson')
        self.reaches = []
        self.ds_reaches = None
        self.impact = None
        self.not_impacted = None
        self.cl = None
        
        self.fig = None
        self.iline1 = None
        self.iline2 = None
        self.loc_point = None
        self.ilines = []
        self.costax = None
        
        self.init_plot()
        
    def cost_calc(self):
        if self.last_pick == None:
            return
        self.costax.clear()
        
        self.costax.text(0.5,1.01,'Hydromod Offset Costs',ha='center',va='bottom',transform=self.costax.transAxes,fontweight='bold',fontsize=14)
        self.costax.text(0.5,0.9,'Q2 Increase = {0} cfs'.format(round(self.Q,3)),ha='center',transform=self.costax.transAxes)
        self.costax.text(0.5,0.83,'Impervious Area Increase = {0} acres'.format(round(self.I,3)),ha='center',transform=self.costax.transAxes)
        self.costax.text(0.5,0.7,'Base fee determined by\nnew impervious area',fontweight='bold',ha='center',va='center',transform=self.costax.transAxes)
        self.costax.text(0.5,0.6,'Total DS Cost = ${0} MM'.format(round(self.ds_reaches.cost.sum()/1000000,1)),ha='center',transform=self.costax.transAxes)
        self.costax.text(0.5,0.53,'Base Fee = ${0}'.format(int(round(self.ds_reaches.fee.sum(),-2))),ha='center',transform=self.costax.transAxes)
        self.costax.text(0.5,0.35,'High impact fee determined by\noffsite Q2 discharge',fontweight='bold',ha='center',va='center',transform=self.costax.transAxes)
        self.costax.text(0.5,0.25,'High Impact Threshold = {0}% of $\Delta$Q2'.format(round(self.impact_threshold,3)),ha='center',transform=self.costax.transAxes)
        self.costax.text(0.5,0.18,'High Impact Distance = {0} miles'.format(round(self.impacted.geometry.length.sum()/5280,2)),ha='center',transform=self.costax.transAxes)
        self.costax.text(0.5,0.11,'High Impact Fee = ${0}'.format(int(round((self.impacted.cost*(self.impacted.Q_diff_percent/100)).sum(),-2))),ha='center',transform=self.costax.transAxes)    

        self.fig.canvas.draw()
            
    def get_impact(self):
        if not self.last_pick:
            return
        #self.ds_reaches['Q_diff'] = self.ds_reaches.Q2_PC - self.ds_reaches.Q2_EC
        #self.ds_reaches['Q_diff_percent'] = self.Q/self.ds_reaches.Q_diff*100
        self.ds_reaches['Q_diff_percent'] = self.Q/self.ds_reaches.Q2_EC*100

        self.ds_reaches['area'] = self.ds_reaches['Wvw_PC'] * self.ds_reaches.geometry.length / 43560
        self.ds_reaches['imp_avail'] = (self.ds_reaches.DA*self.ds_reaches['US_IMP_%_P']/100) - (self.ds_reaches.DA*self.ds_reaches['US_IMP_%_E']/100)
        self.ds_reaches['fee'] = (self.I/640/self.ds_reaches.imp_avail) * self.ds_reaches.cost

        self.impacted = self.ds_reaches[self.ds_reaches.Q_diff_percent>=self.impact_threshold]
        self.not_impacted = self.ds_reaches[self.ds_reaches.Q_diff_percent<self.impact_threshold]

        if len(self.ilines)>0:
            for art in self.ilines:
                art.remove()
            self.ilines = []
                
        if not self.impacted.empty:
            self.impact_dist = self.impacted['dist_from_pp'].values[-1]
            if self.iline1:
                self.iline1.remove()
                self.iline2.remove()
                self.iline1 = self.ax1.axvspan(0,self.impact_dist,color='yellow',alpha=0.15)
                self.iline2 = self.ax12.axvspan(0,self.impact_dist,color='yellow',alpha=0.15)
            else:
                self.iline1 = self.ax1.axvspan(0,self.impact_dist,color='yellow',alpha=0.15)
                self.iline2 = self.ax12.axvspan(0,self.impact_dist,color='yellow',alpha=0.15)
            
            xys = self.impacted.geometry.apply(lambda x: np.array(x.coords))
            for xy in xys:
                l, = self.ax0.plot(xy[:,0],xy[:,1],lw=5,color='yellow',alpha=0.5,zorder=10)
                self.ilines.append(l)
                
        self.cost_calc()
        
    def init_plot(self):

        self.df.crs = 'EPSG:3644'
        self.df.DS_reach = self.df.DS_reach.astype('Int64')
        #self.df.ds_reach_l = self.df.ds_reach_l.apply(lambda x: [int(j) for j in x.split(',')])
        
        self.fig = plt.figure()
        gs = gridspec.GridSpec(3, 3, width_ratios=[2,2,1], height_ratios=[2,1,1])
        self.ax0 = self.fig.add_subplot(gs[0, 0])
        self.ax1 = self.fig.add_subplot(gs[1, :])
        ax2 = self.fig.add_subplot(gs[0, 1])
        self.costax = self.fig.add_subplot(gs[0,2])
        self.ax12 = self.fig.add_subplot(gs[2,:])
        #self.ax12.set_yscale('log')
        self.ax13 = self.ax12.twinx()
    
        cmap = {'Grading':'peru',
               'BDA':'green',
               'ELJ':'limegreen',
               'Roughen':'fuchsia',
               'Bypass':'k'}
        for i,row in self.df.iterrows():
            r, = self.ax0.plot(row.geometry.coords.xy[0],row.geometry.coords.xy[1],picker=2,color='tab:blue',zorder=100)
            self.reaches.append(r)
            
        ctx.add_basemap(ax=self.ax0,crs=self.df.crs, source=ctx.providers.Esri.WorldImagery, zoom=16)
        ctx.add_basemap(ax=self.ax0,crs=self.df.crs, source=ctx.providers.Esri.WorldShadedRelief, zoom=13, alpha=0.5)

        self.costax.tick_params(
                axis='both',          
                which='both',     
                bottom=False,      
                top=False,        
                labelbottom=False,
                labelleft=False) 
        
        self.ax0.tick_params(
                axis='both',          
                which='both',     
                bottom=False,      
                top=False,        
                labelbottom=False,
                labelleft=False) 
        
        def on_pick(event):              
            if self.last_pick:
                for idx in self.ds_reaches.index:
                    self.reaches[idx].update({'color':'tab:blue'})
                self.last_pick.update({'color':'tab:blue'})
            self.last_pick = event.artist
            
            if self.loc_point:
                self.loc_point.remove()
                self.loc_point = None
            
            idx = self.reaches.index(event.artist)
            ds_idx = self.df.at[idx,'ds_reach_l']
            ds_idx = [int(j) for j in ds_idx.split(',')]
            self.ds_reaches = self.df.iloc[ds_idx,:]
            
            event.artist.update({'color':cmap[self.df.loc[idx,'Treatment']]})
            for idx in self.ds_reaches.index:
                self.reaches[idx].update({'color':cmap[self.df.loc[idx,'Treatment']]})
                            
            self.ax1.clear()
            self.ax12.clear()
            #self.ax12.set_yscale('log')
            self.ax13.clear()
            ax2.clear()
            
            self.ds_reaches = self.ds_reaches.sort_values(by=['Q2_PC'])
            self.cl = self.ds_reaches.geometry.unary_union
            
            self.ds_reaches['dist_from_pp'] = self.ds_reaches.geometry.length.cumsum()/5280
            
            self.ax1.plot(self.ds_reaches.dist_from_pp,self.ds_reaches.Wvw_PC,color='peru',label='Existing Valley Width')
            self.ax1.plot(self.ds_reaches.dist_from_pp,self.ds_reaches.Wvw_thresh,color='peru',ls='--',label='Width Required for RSC')
            self.ax1.legend(loc='upper left')
            self.ax1.grid()
            
            self.ax12.plot(self.ds_reaches.dist_from_pp,self.ds_reaches.Q2_EC,color='tab:blue',label='Existing Q2')
            self.ax12.plot(self.ds_reaches.dist_from_pp,self.ds_reaches.Q2_PC,color='tab:blue',ls='--',label='Future Q2')
            self.ax12.plot(self.ds_reaches.dist_from_pp,self.ds_reaches.Q2_nat,color='tab:blue',ls=':',label='Natural Q2')
            self.ax12.fill_between(self.ds_reaches.dist_from_pp,self.ds_reaches.Q2_nat,self.ds_reaches.Q2_PC,color='tab:blue',alpha=0.15)
            self.ax12.legend(loc='lower left')
            self.ax12.grid()
            self.ax13.plot(self.ds_reaches.dist_from_pp,100-self.ds_reaches['US_IMP_%_E'],color='k',ls='--',label='Existing Undeveloped Area')
            #self.ax13.plot(self.ds_reaches.dist_from_pp,100-self.ds_reaches['US_IMP_%_P'],color='k',ls='--',alpha=0.5,label='Future Development')
            self.ax13.legend(loc='lower right')
            self.ax13.set_ylim(0,100)
            
            self.get_impact()
           
            ds_reaches_groups = self.ds_reaches.groupby('Treatment')[['cost','area']].sum().reset_index()
            (ds_reaches_groups['cost']/1000000).plot.barh(ax=ax2,color=ds_reaches_groups.Treatment.map(cmap))
            
            for i,row in ds_reaches_groups.iterrows():
                ax2.text(row.cost/1000000+0.01,i,str(round(row.area,1))+' acres',ha='left')
            ax2.set_yticklabels(ds_reaches_groups.Treatment)
            ax2.set_xlim(0,ax2.get_xlim()[1]*1.15)
            ax2.set_xlabel('Downstream Stream Enhancement Cost ($MM)')
                            
            #self.ax12.set_ylabel('Discharge (cfs)', color='tab:blue')
            self.ax12.set_ylabel('Discharge (cfs)', color='tab:blue')
            self.ax12.spines['right'].set_position(('axes', 1.05))
            self.ax12.spines['right'].set_color('tab:blue')
            self.ax12.tick_params(axis='y', colors='tab:blue')
            self.ax13.set_ylabel('Undeveloped Area (%)')
            self.ax1.set_ylabel('Valley Width (ft)',color='peru')
            self.ax1.spines['left'].set_color('peru')
            self.ax1.tick_params(axis='y', colors='peru')
            self.ax12.set_xlabel('Distance Downstream (miles)')
            
            self.cost_calc()
            
            self.fig.canvas.draw()
            gs.update(left=0.04)
            gs.update(right=0.96)
            gs.update(top=0.96)
            gs.update(bottom=0.06)
            gs.update(wspace=0.14)
            gs.update(hspace=0.14)
            
        def on_click(event):
            if not self.cl:
                return
            if event.inaxes not in [self.ax1,self.ax12,self.ax13]:
                return
            
            dist = event.xdata*5280
            point = self.cl.interpolate(dist)
            if not self.loc_point:
                self.loc_point = self.ax0.scatter(point.x,point.y,color='k')
            else:
                self.loc_point.set(offsets=(point.x,point.y))
                
            self.fig.canvas.draw()

        self.fig.canvas.mpl_connect('button_press_event', on_click)
        self.fig.canvas.mpl_connect('pick_event', on_pick)

In [139]:
app = Application()

In [140]:
app.Q = 0.2
app.I = 1
app.impact_threshold = 1

app.get_impact()

In [ ]:
#fig,ax = plt.subplots(1,1)
#rng = np.arange(0,101,0.01)
#ax.plot(rng,rng**0.3)
#ax.set_xlabel('Percent Impervious Area')
#ax.set_ylabel('Discharge Multiplier')
#ax.grid()